Steps:
1. Import libraries 
2. Load dataset
3. Make a copy of original dataset and use one for continued pre-processing
4. Inspect the data | Check for inconsistencies and missing values, duplicates
5. Split the data set in train/test
6. Separate the categorical and numerical features
7. Check for missing and handle missing values in categorical and numerical using simple imputer
8. Fit model and evaluate it

# Step 1: Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
set_config(display='diagram')

# Step 2: Load Dataset

In [2]:
path = './Fish - Fish.csv'
fish_df = pd.read_csv(path)
fish_df.info()
fish_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Species  155 non-null    object 
 1   Weight   159 non-null    float64
 2   Length1  157 non-null    float64
 3   Length2  157 non-null    float64
 4   Length3  150 non-null    float64
 5   Height   156 non-null    float64
 6   Width    157 non-null    float64
dtypes: float64(6), object(1)
memory usage: 8.8+ KB


,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [3]:
fish_df_copy = fish_df.copy()

# Step 4: Inspect the Data

    There are 0 duplicates.  However, there seems to be some null values present based on initial observance of dataset

In [4]:
fish_df.shape

(159, 7)

In [5]:
# reorder columns so that it is easier to search for target 
column_reordered = ['Species', 'Length1', 'Length2', 'Length3', 'Height', 'Width', 'Weight']

fish_df = fish_df[column_reordered]

In [6]:
fish_df.head()

,Species,Length1,Length2,Length3,Height,Width,Weight
0,Bream,23.2,25.4,30.0,11.5200,4.0200,242.0
1,Bream,24.0,26.3,31.2,12.4800,4.3056,290.0
2,Bream,23.9,26.5,31.1,12.3778,4.6961,340.0
3,Bream,26.3,29.0,33.5,12.7300,4.4555,363.0
4,Bream,26.5,29.0,34.0,12.4440,5.1340,430.0


159 rows, 9 columns

In [10]:
def chk_for_dups_null(df):
    print("checking for duplicates ===>  \n",df.duplicated().sum())
    print("checking for null values ===> \n",df.isna().sum())
chk_for_dups_null(fish_df)


checking for duplicates ===>  
 0
checking for null values ===> 
 Species    4
Length1    2
Length2    2
Length3    9
Height     3
Width      2
Weight     0
dtype: int64


However, there are null values. We could drop them since they're 5% less of the dataframe, but we are practicing pipelines and we will create a pipeline for filling in missing data and standardizing.

# Step 5: Split the Dataset

    Why split data:
            Training Data: This is the portion of your data used to train your machine learning model. The model learns patterns, relationships, and rules from this data.
            Testing Data: This is a separate portion of your data that the model has never seen during training. It's used to assess how well the trained model generalizes to new, unseen data.

In [11]:
target = 'Weight'
X = fish_df.drop(columns=[target])
y= fish_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    The objects returned by the train_test_split function are typically NumPy arrays or pandas DataFrames (depending on the data types of the input data). These objects are also considered data structures or data containers.
    
    X_train, X_test, y_train, and y_test are data containers (objects) that hold your feature and target data. The specific data type of these objects depends on the data types of your input data when calling train_test_split.
    
    X_train: Contains the feature data for the training set. It's typically a DataFrame or NumPy array. It's the portion of your data that your machine learning model will be trained on.

    X_test: Contains the feature data for the testing set. It's also typically a DataFrame or NumPy array. It's used to evaluate the model's performance on unseen data.

    y_train: Contains the target variable data for the training set. It's typically a Series (if using pandas) or a NumPy array.

    y_test: Contains the target variable data for the testing set. It's also typically a Series (if using pandas) or a NumPy array.

    The specific order of these variables (X_train, X_test, y_train, y_test) is a convention often used in machine learning to make it clear which sets correspond to feature data and target data. It's also the order in which the train_test_split function returns the values.

# Step 6 : Separate categorical and numerical data 

In [16]:
# instantiate the columns selectors
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

num_selector and cat_selector grabs all the columns that have the corresponding data types for "number" or "categorical" aka "object"

,Species
0,Bream
1,Bream
2,Bream
3,Bream
4,Bream
...,...
154,Smelt
155,Smelt
156,Smelt
157,Smelt
